# LD_CCSAQ algorithm:
- Convex subproblem at each iteration of the inner loop
- Each subproblem is the combination of linear Taylor expansion of the objective and constraint function (separable functions)
- The quadratic part is a quadratic penalty term that ensures that the next step in the outer loop is conservative.
- Each steps of the outer loop requires the evaluation of the functions + gradient
- Inside the inner loop just need the functions evaluations.

In [1]:
#example from https://github.com/JuliaOpt/NLopt.jl

using NLopt

count = 0 # keep track of # function evaluations

function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end

    global count
    count::Int += 1
    
    f = sqrt(x[2])
    println("f_$count($x)=$f")

    f
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end
    (a*x[1] + b)^3 - x[2]
end

opt = Opt(:LD_CCSAQ, 2)
lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt,1e-4)

min_objective!(opt, myfunc)
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,2,0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x,g,-1,1), 1e-8)

(minf,minx,ret) = optimize(opt, [1.234, 5.678])
println("got $minf at $minx after $count iterations (returned $ret)")

f_1([1.234,5.678])=2.382855429941145
f_2([0.9710289795117957,5.4768731703791955])=2.340272029140885
f_3([0.8694775756072178,5.277298044043508])=2.297237045679768
f_4([0.7876540847066766,4.277298044043508])=2.0681629636088905
f_5([0.7284112027079249,3.0772980440435083])=1.7542229174319632
f_6([0.7269979172024564,3.0772980440435083])=1.7542229174319632
f_7([0.6131801133176963,1.6372980440435083])=1.279569476051812
f_8([0.608109251634216,1.6372980440435083])=1.279569476051812
f_9([0.5934863218552073,1.6372980440435083])=1.279569476051812
f_10([0.5884060343887269,1.6372980440435083])=1.279569476051812
f_11([0.40896850238057525,0.16229380405800797])=0.40285705164240077
f_12([0.3870088158974541,0.18540814499939517])=0.4305904608783097
f_13([0.35958624504984993,0.24536531419947716])=0.4953436324406292
f_14([0.3434232328439481,0.27535608494126573])=0.5247438279210779
f_15([0.33346936910355085,0.29458604746579753])=0.5427578165865486
f_16([0.3270125193071079,0.30732704024387325])=0.554370850824

# Discussion
We see that the 19th evaluation of the function is actually lower than the final value. This must have happened in an inner loop where the local optimum of the convex problem did not satisfy the conservative property, so it had to increase the penalty term at the function approximation level.